In [7]:
import vtk
import numpy as np
from scipy.interpolate import splprep, splev
from scipy.spatial import cKDTree

# ============ 通用函数 ============

def load_polyline_from_vtk(fname):
    """读取 VTK polyline，返回 Nx3 numpy 数组（毫米单位）"""
    reader = vtk.vtkPolyDataReader()
    reader.SetFileName(fname)
    reader.Update()
    polydata = reader.GetOutput()
    points = polydata.GetPoints()
    return np.array([points.GetPoint(i) for i in range(points.GetNumberOfPoints())])

def cumulative_arc_length(P):
    seg = np.linalg.norm(np.diff(P, axis=0), axis=1)
    s = np.concatenate([[0.0], np.cumsum(seg)])
    return s, s[-1]

def resample_polyline_equal_arclength(P, M):
    s, L = cumulative_arc_length(P)
    if L == 0:
        return np.repeat(P[:1], M, axis=0)
    s_target = np.linspace(0.0, L, M)
    idx = np.searchsorted(s, s_target, side='right') - 1
    idx = np.clip(idx, 0, len(s)-2)
    t = (s_target - s[idx]) / (s[idx+1] - s[idx] + 1e-12)
    return P[idx] + (P[idx+1] - P[idx]) * t[:, None]

def build_param_to_arclen_map_spline(tck, n_dense=5000, t_min=None, t_max=None):
    if t_min is None or t_max is None:
        t_min, t_max = tck[0][0], tck[0][-1]
    t_dense = np.linspace(t_min, t_max, n_dense)
    xyz = np.vstack(splev(t_dense, tck)).T
    s, L = cumulative_arc_length(xyz)
    s_norm = s / (L + 1e-12)
    return (t_dense, s_norm, L)

def invert_s_to_t(s_norm_grid, t_dense, s_norm_dense):
    idx = np.searchsorted(s_norm_dense, s_norm_grid, side='right') - 1
    idx = np.clip(idx, 0, len(s_norm_dense)-2)
    w = (s_norm_grid - s_norm_dense[idx]) / (s_norm_dense[idx+1] - s_norm_dense[idx] + 1e-12)
    return t_dense[idx] + (t_dense[idx+1] - t_dense[idx]) * w

def resample_spline_equal_arclength(tck, M, t_min=None, t_max=None):
    t_dense, s_norm_dense, L = build_param_to_arclen_map_spline(tck, t_min, t_max)
    s_norm_target = np.linspace(0.0, 1.0, M)
    t_target = invert_s_to_t(s_norm_target, t_dense, s_norm_dense)
    return np.vstack(splev(t_target, tck)).T

def rmse_points(P, Q):
    return np.sqrt(np.mean(np.sum((P - Q)**2, axis=1)))

# ============ 方法 A：按弧长对齐采样 RMSE ============
def rmse_poly_vs_spline_equal_arclength(P_orig, tck_spline, M=1000):
    P_res = resample_polyline_equal_arclength(P_orig, M)
    S_res = resample_spline_equal_arclength(tck_spline, M)
    return rmse_points(P_res, S_res)

# ============ 方法 B：Chamfer RMSE ============
def chamfer_rmse(P, Q):
    kd_P, kd_Q = cKDTree(P), cKDTree(Q)
    dPQ, _ = kd_Q.query(P, k=1)
    dQP, _ = kd_P.query(Q, k=1)
    return np.sqrt(0.5*(np.mean(dPQ**2) + np.mean(dQP**2)))

def chamfer_rmse_poly_vs_spline(P_orig, tck_spline, M_poly=1500, M_spline=3000):
    P_dense = resample_polyline_equal_arclength(P_orig, M_poly)
    S_dense = resample_spline_equal_arclength(tck_spline, M_spline)
    return chamfer_rmse(P_dense, S_dense)

def normalize_to_origin(P):
    """把曲线 P (N,3) 平移，使起点在原点"""
    return P - P[0]

# ============ 主调用 ============
if __name__ == "__main__":
    fname_orig = r"D:\!BraVa_src\Brava\major\BG0002.CNG.swc.vtk_Left.vtk"
    fname_spline = r"D:\!BraVa_src\Brava\splined_major\result\BG0002_Left.vtk"

    # 加载两条曲线
    P_orig = load_polyline_from_vtk(fname_orig)
    P_spline = load_polyline_from_vtk(fname_spline)

    # 平移到原点
    P_orig = normalize_to_origin(P_orig)
    P_spline = normalize_to_origin(P_spline)

    # 方法 A：等弧长 RMSE
    P_res = resample_polyline_equal_arclength(P_orig, M=240)
    S_res = resample_polyline_equal_arclength(P_spline, M=240)
    rmse_A = rmse_points(P_res, S_res)
    print("方法 A (等弧长 RMSE):", rmse_A, "mm")

    # 方法 B：Chamfer RMSE
    rmse_B = chamfer_rmse(P_res, S_res)
    print("方法 B (Chamfer RMSE):", rmse_B, "mm")




方法 A (等弧长 RMSE): 0.620903010820392 mm
方法 B (Chamfer RMSE): 0.37329559516567584 mm


In [9]:
import os
import re
import numpy as np

def normalize_key_from_orig(fname):
    # e.g. BG0002.CNG.swc.vtk_Left.vtk -> BG2_Left
    m = re.match(r"(B[A-Z])0*([0-9]+)\.CNG\.swc\.vtk_(Left|Right)\.vtk", fname)
    if not m:
        return None
    return f"{m.group(1)}{int(m.group(2))}_{m.group(3)}"

def normalize_key_from_spline(fname):
    # e.g. BG0002_Left.vtk -> BG2_Left
    m = re.match(r"(B[A-Z])0*([0-9]+)_(Left|Right)\.vtk", fname)
    if not m:
        return None
    return f"{m.group(1)}{int(m.group(2))}_{m.group(3)}"

# 假设这两个函数已经有了 RMSE 计算逻辑
def compute_rmse_for_pair(fname_orig, fname_spline):
    P_orig = load_polyline_from_vtk(fname_orig)
    P_spline = load_polyline_from_vtk(fname_spline)
    P_orig = normalize_to_origin(P_orig)
    P_spline = normalize_to_origin(P_spline)
    P_res = resample_polyline_equal_arclength(P_orig, M=240)
    S_res = resample_polyline_equal_arclength(P_spline, M=240)
    return rmse_points(P_res, S_res)

def batch_rmse(dir_orig, dir_spline):
    # 建立映射表
    orig_map = {}
    for f in os.listdir(dir_orig):
        key = normalize_key_from_orig(f)
        if key: orig_map[key] = os.path.join(dir_orig, f)

    spline_map = {}
    for f in os.listdir(dir_spline):
        key = normalize_key_from_spline(f)
        if key: spline_map[key] = os.path.join(dir_spline, f)

    # 遍历匹配
    rmses = []
    for key, f_orig in orig_map.items():
        if key in spline_map:
            f_spline = spline_map[key]
            rmse_val = compute_rmse_for_pair(f_orig, f_spline)
            print(f"{key}: RMSE = {rmse_val:.3f} mm")
            rmses.append(rmse_val)

    if rmses:
        print("全体均值 RMSE:", np.mean(rmses), "mm")
        print("全体方差 RMSE:", np.var(rmses, ddof=1), "mm^2")
    else:
        print("未找到匹配文件")

# 调用示例
dir_orig   = r"D:\!BraVa_src\Brava\major"
dir_spline = r"D:\!BraVa_src\Brava\splined_major\result"
batch_rmse(dir_orig, dir_spline)


BG2_Left: RMSE = 0.621 mm
BG2_Right: RMSE = 0.599 mm
BG14_Left: RMSE = 0.801 mm
BG14_Right: RMSE = 0.732 mm
BG19_Left: RMSE = 0.970 mm
BG19_Right: RMSE = 0.828 mm
BG20_Left: RMSE = 1.387 mm
BG20_Right: RMSE = 1.139 mm
BG21_Left: RMSE = 0.921 mm
BG21_Right: RMSE = 0.684 mm
BG22_Left: RMSE = 1.094 mm
BG22_Right: RMSE = 1.357 mm
BG4_Left: RMSE = 0.970 mm
BG4_Right: RMSE = 0.719 mm
BG9_Left: RMSE = 0.991 mm
BG9_Right: RMSE = 1.000 mm
BG10_Left: RMSE = 0.850 mm
BG10_Right: RMSE = 1.088 mm
BG11_Left: RMSE = 0.795 mm
BG11_Right: RMSE = 1.054 mm
BG12_Left: RMSE = 0.848 mm
BG12_Right: RMSE = 1.382 mm
BG13_Left: RMSE = 0.622 mm
BG13_Right: RMSE = 0.715 mm
BG15_Left: RMSE = 0.985 mm
BG15_Right: RMSE = 0.592 mm
BG17_Left: RMSE = 1.046 mm
BG17_Right: RMSE = 1.334 mm
BG18_Left: RMSE = 1.306 mm
BG18_Right: RMSE = 1.033 mm
BH2_Left: RMSE = 0.501 mm
BH2_Right: RMSE = 0.695 mm
BH3_Left: RMSE = 0.945 mm
BH3_Right: RMSE = 0.947 mm
BH4_Left: RMSE = 0.395 mm
BH4_Right: RMSE = 0.484 mm
BH5_Left: RMSE = 1.290